<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_NEWAGENT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://mistral.ai/news/agents-api

https://console.mistral.ai/usage

## MISTRAL SETUP

In [ ]:
!pip install colab-env --quiet
!pip install mistralai --quiet

In [ ]:
from IPython import get_ipython
from IPython.display import display
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env

In [3]:
from mistralai import Mistral
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)


client = Mistral(api_key=api_key)  # Use MistralClient instead of Mistral
model_list = client.models.list()

for model in model_list.data:
    print(model.id)
    #print(model.created)
    #print(model.owned_by)

mistral-medium-2505
mistral-medium-latest
mistral-medium
ministral-3b-2410
ministral-3b-latest
ministral-8b-2410
ministral-8b-latest
open-mistral-7b
mistral-tiny
mistral-tiny-2312
open-mistral-nemo
open-mistral-nemo-2407
mistral-tiny-2407
mistral-tiny-latest
open-mixtral-8x7b
mistral-small
mistral-small-2312
open-mixtral-8x22b
open-mixtral-8x22b-2404
mistral-small-2409
mistral-large-2407
mistral-large-2411
mistral-large-latest
pixtral-large-2411
pixtral-large-latest
mistral-large-pixtral-2411
codestral-2501
codestral-latest
codestral-2412
codestral-2411-rc5
devstral-small-2505
devstral-small-latest
pixtral-12b-2409
pixtral-12b
pixtral-12b-latest
mistral-small-2501
mistral-small-2503
mistral-small-2506
mistral-small-latest
mistral-saba-2502
mistral-saba-latest
magistral-medium-2506
magistral-medium-latest
magistral-small-2506
magistral-small-latest
mistral-embed
codestral-embed
codestral-embed-2505
mistral-moderation-2411
mistral-moderation-latest
mistral-ocr-2503
mistral-ocr-2505
mistr

## FLIGHTS SEARCH

In [15]:
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env --quiet
    import colab_env

try:
    import mistralai
    # Re-import after upgrade to ensure new version is loaded in current session if needed
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai --upgrade'.")
    exit()

# %%
# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# --- Agent Definitions ---
# Pydantic model for Calculator Agent's response format (still relevant if that agent is used)
class CalcResult(BaseModel):
    reasoning: str
    result: str

print("Creating AI agents...")

finance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent used to answer financial related requests",
    name="finance-agent",
)
web_search_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent that can search online for any information if needed",
    name="websearch-agent",
    tools=[{"type": "web_search"}],
)
ecb_interest_rate_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Can find the current interest rate of the European central bank",
    name="ecb-interest-rate-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_european_central_bank_interest_rate",
                "description": "Retrieve the real interest rate of European central bank.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                        },
                    },
                    "required": [
                        "date",
                    ]
                },
            },
        },
    ],
)
graph_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="graph-drawing-agent",
    description="Agent used to create graphs using the code interpreter tool.",
    instructions="Use the code interpreter tool when you have to draw a graph.",
    tools=[{"type": "code_interpreter"}]
)
calculator_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="calculator-agent",
    description="Agent used to make detailed calculations",
    instructions="When doing calculations explain step by step what you are doing.",
    completion_args={
          "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "calc_result",
                "schema": CalcResult.model_json_schema(),
            }
        }
    }
)

# --- Custom function for flight search (MOCK) ---
# This is a LOCAL MOCK FUNCTION.
# When the remote Mistral AI model proposes to use this tool,
# this local Python function needs to be executed by your application.
# For a real integration, this functionality would typically be
# an actual web service endpoint accessible by Mistral's platform.
def search_flights(origin: str, destination: str, departure_date: str, return_date: str = None):
    """
    MOCK: This function simulates searching for available flights.
    It returns dummy data based on specific input parameters.
    """
    print(f"\n[DEBUG] MOCK CALL: search_flights with origin='{origin}', destination='{destination}', departure_date='{departure_date}', return_date='{return_date}')")

    origin_norm = origin.upper() if origin else ""
    destination_norm = destination.upper() if destination else ""

    # Dummy data for demonstration
    if origin_norm == "YUL" and destination_norm == "JFK" and departure_date == "2025-07-01":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "AC700", "airline": "Air Canada", "departure_time": "10:00 AM", "arrival_time": "11:30 AM", "price": "$250 CAD"},
                {"flight_number": "DL123", "airline": "Delta Airlines", "departure_time": "11:00 AM", "arrival_time": "12:45 PM", "price": "$280 CAD"}
            ]
        }
    elif origin_norm == "LAX" and destination_norm == "SFO" and departure_date == "2025-08-15":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "UA456", "airline": "United Airlines", "departure_time": "09:00 AM", "arrival_time": "10:15 AM", "price": "$120 USD"},
            ]
        }
    elif origin_norm == "YUL" and destination_norm in ["HND", "NRT"]: # Assuming HND or NRT for Tokyo
        return {
            "flights_found": True,
            "details": [
                 {"flight_number": "AC005", "airline": "Air Canada", "departure_time": "01:45 PM", "arrival_time": "03:30 PM +1 day", "price": "$1200 CAD"},
                 {"flight_number": "NH117", "airline": "ANA", "departure_time": "04:00 PM", "arrival_time": "05:50 PM +1 day", "price": "$1350 CAD"}
            ]
        }
    # Add new dummy data for a different route/date
    elif origin_norm == "LHR" and destination_norm == "CDG" and departure_date == "2025-09-10":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "BA123", "airline": "British Airways", "departure_time": "08:00 AM", "arrival_time": "10:30 AM", "price": "£150 GBP"},
                {"flight_number": "AF456", "airline": "Air France", "departure_time": "09:00 AM", "arrival_time": "11:45 AM", "price": "€170 EUR"}
            ]
        }
    else:
        return {"flights_found": False, "details": "No flights found for the specified criteria."}

# --- Flight Planning Agent Definition ---
# The agent is defined with its capabilities and tools on the Mistral platform.
flight_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting with flight planning requests, searching for flights and providing relevant information. Use IATA airport codes for origin and destination if possible. For example, Montreal is YUL and New York is JFK or LGA.",
    name="flight-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_flights",
                "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                        "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                        "departure_date": {"type": "string", "description": "Departure date inYYYY-MM-DD format"},
                        "return_date": {"type": "string", "description": "Return date inYYYY-MM-DD format (optional)"}
                    },
                    "required": ["origin", "destination", "departure_date"]
                }
            }
        },
        # For agent creation, this simple format for web_search is usually correct.
        {"type": "web_search"}
    ]
)

print(f"\nFlight Planning Agent '{flight_planning_agent.name}' created with ID: {flight_planning_agent.id}")
tool_names_list = []
for tool in flight_planning_agent.tools:
    if tool.type == 'function':
        tool_names_list.append(tool.function.name)
    else:
        tool_names_list.append(tool.type)
print(f"Tools available to flight_planning_agent: {tool_names_list}")


# --- Test Case Execution for Flight Planning Agent ---
print("\n--- Executing Test Cases for the Flight Planning Agent (Consecutive) ---")
print("This simulates multi-turn conversations where your code acts as the tool executor.")

# Manual construction of tools list for the chat.complete call (remains the same for both tests)
api_call_tools_list = [
    {
        "type": "function",
        "function": {
            "name": "search_flights",
            "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                    "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                    "departure_date": {"type": "string", "description": "Departure date inYYYY-MM-DD format"},
                    "return_date": {"type": "string", "description": "Return date inYYYY-MM-DD format (optional)"}
                },
                "required": ["origin", "destination", "departure_date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "internal_web_search_tool",
            "description": "Accesses the internet to find information.",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    }
]

# --- Test Case 1: YUL to NRT ---
print("\n--- Test Case 1: YUL to NRT on July 10th, 2025 ---")
user_query_1 = "Find me a flight from Montreal (YUL) to Tokyo on July 10th, 2025."
print(f"\nUser: {user_query_1}")
conversation_history_1 = [{"role": "user", "content": user_query_1}]

try:
    print("[DEBUG] Sending initial user query 1 to the agent...")
    response_turn1_1 = client.chat.complete(
        model=flight_planning_agent.model,
        messages=conversation_history_1,
        tools=api_call_tools_list,
    )
    assistant_message_turn1_1 = response_turn1_1.choices[0].message
    conversation_history_1.append(assistant_message_turn1_1.model_dump() if hasattr(assistant_message_turn1_1, 'model_dump') else assistant_message_turn1_1.__dict__)

    if hasattr(assistant_message_turn1_1, 'tool_calls') and assistant_message_turn1_1.tool_calls:
        print("\nAgent proposed tool calls (Turn 1, Test 1):")
        for tool_call in assistant_message_turn1_1.tool_calls:
            print(f"  Tool Name: {tool_call.function.name}")
            print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")
            tool_output_content = None
            if tool_call.function.name == "search_flights":
                try:
                    args = json.loads(tool_call.function.arguments)
                    tool_output = search_flights(**args)
                    tool_output_content = json.dumps(tool_output)
                    print(f"  [DEBUG] Local MOCK search_flights executed. Output: {tool_output_content}")
                except json.JSONDecodeError as e:
                    print(f"  [ERROR] Failed to parse tool arguments: {e}")
                    tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                except Exception as e:
                    print(f"  [ERROR] Error executing local mock search_flights: {e}")
                    tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
            elif tool_call.function.name == "internal_web_search_tool":
                print(f"  [DEBUG] Web search requested by agent. Providing mock output for local simulation.")
                tool_output_content = json.dumps("Mock web search: Information related to flight search.")
            else:
                print(f"  [DEBUG] Unhandled tool call: {tool_call.function.name}")
                tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

            conversation_history_1.append(
                {
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": tool_output_content,
                    "tool_call_id": tool_call.id
                }
            )
            print(f"  [DEBUG] Tool output for '{tool_call.function.name}' added to history 1.")

        print("\n[DEBUG] Sending conversation history 1 with tool outputs back for final response...")
        final_response_1 = client.chat.complete(
            model=flight_planning_agent.model,
            messages=conversation_history_1,
            tools=api_call_tools_list,
        )
        final_assistant_message_1 = final_response_1.choices[0].message
        print("\nAgent's Final Response (Test 1):")
        print(final_assistant_message_1.content)
        conversation_history_1.append(final_assistant_message_1.model_dump() if hasattr(final_assistant_message_1, 'model_dump') else assistant_message_turn1_1.__dict__)
    else:
        print("\nAgent's initial response (no tool calls proposed, Test 1):")
        print(assistant_message_turn1_1.content)

except Exception as e:
    print(f"\nAn error occurred during agent interaction (Test 1): {e}")


print("\n--- Test Case 1 Complete ---")

# --- Test Case 2: LHR to CDG ---
print("\n--- Test Case 2: LHR to CDG on September 10th, 2025 ---")
user_query_2 = "Find me a flight from London (LHR) to Paris (CDG) on September 10th, 2025."
print(f"\nUser: {user_query_2}")
conversation_history_2 = [{"role": "user", "content": user_query_2}] # Start new history for Test 2

try:
    print("[DEBUG] Sending initial user query 2 to the agent...")
    response_turn1_2 = client.chat.complete(
        model=flight_planning_agent.model,
        messages=conversation_history_2,
        tools=api_call_tools_list,
    )
    assistant_message_turn1_2 = response_turn1_2.choices[0].message
    conversation_history_2.append(assistant_message_turn1_2.model_dump() if hasattr(assistant_message_turn1_2, 'model_dump') else assistant_message_turn1_2.__dict__)

    if hasattr(assistant_message_turn1_2, 'tool_calls') and assistant_message_turn1_2.tool_calls:
        print("\nAgent proposed tool calls (Turn 1, Test 2):")
        for tool_call in assistant_message_turn1_2.tool_calls:
            print(f"  Tool Name: {tool_call.function.name}")
            print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")
            tool_output_content = None
            if tool_call.function.name == "search_flights":
                try:
                    args = json.loads(tool_call.function.arguments)
                    tool_output = search_flights(**args)
                    tool_output_content = json.dumps(tool_output)
                    print(f"  [DEBUG] Local MOCK search_flights executed. Output: {tool_output_content}")
                except json.JSONDecodeError as e:
                    print(f"  [ERROR] Failed to parse tool arguments: {e}")
                    tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                except Exception as e:
                    print(f"  [ERROR] Error executing local mock search_flights: {e}")
                    tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
            elif tool_call.function.name == "internal_web_search_tool":
                print(f"  [DEBUG] Web search requested by agent. Providing mock output for local simulation.")
                tool_output_content = json.dumps("Mock web search: Information related to flight search.")
            else:
                print(f"  [DEBUG] Unhandled tool call: {tool_call.function.name}")
                tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

            conversation_history_2.append(
                {
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": tool_output_content,
                    "tool_call_id": tool_call.id
                }
            )
            print(f"  [DEBUG] Tool output for '{tool_call.function.name}' added to history 2.")


        print("\n[DEBUG] Sending conversation history 2 with tool outputs back for final response...")
        final_response_2 = client.chat.complete(
            model=flight_planning_agent.model,
            messages=conversation_history_2,
            tools=api_call_tools_list,
        )
        final_assistant_message_2 = final_response_2.choices[0].message
        print("\nAgent's Final Response (Test 2):")
        print(final_assistant_message_2.content)
        conversation_history_2.append(final_assistant_message_2.model_dump() if hasattr(final_assistant_message_2, 'model_dump') else final_assistant_message_2.__dict__)
    else:
        print("\nAgent's initial response (no tool calls proposed, Test 2):")
        print(assistant_message_turn1_2.content)

except Exception as e:
    print(f"\nAn error occurred during agent interaction (Test 2): {e}")


print("\n--- Test Case 2 Complete ---")

print("\n--- All Consecutive Test Cases Complete ---")

Creating AI agents...

Flight Planning Agent 'flight-planning-agent' created with ID: ag_06861edc1e3c75ba8000c97d61d70a8d
Tools available to flight_planning_agent: ['search_flights', 'web_search']

--- Executing Test Cases for the Flight Planning Agent (Consecutive) ---
This simulates multi-turn conversations where your code acts as the tool executor.

--- Test Case 1: YUL to NRT on July 10th, 2025 ---

User: Find me a flight from Montreal (YUL) to Tokyo on July 10th, 2025.
[DEBUG] Sending initial user query 1 to the agent...

Agent proposed tool calls (Turn 1, Test 1):
  Tool Name: search_flights
  Tool Arguments (JSON string): {"origin": "YUL", "destination": "NRT", "departure_date": "2025-07-10"}

[DEBUG] MOCK CALL: search_flights with origin='YUL', destination='NRT', departure_date='2025-07-10', return_date='None')
  [DEBUG] Local MOCK search_flights executed. Output: {"flights_found": true, "details": [{"flight_number": "AC005", "airline": "Air Canada", "departure_time": "01:45 PM

## **Reasoning**:
Define the calculator query, create a new conversation history, and call the chat.complete method for the calculator agent.



In [17]:
new_calculator_agent = client.beta.agents.create(
    #model="mistral-large-latest",
    model="magistral-medium-latest",
    name="calculator-agent",
    description="Agent used to make detailed calculations",
    instructions="When doing calculations explain step by step what you are doing.",
    completion_args={
          "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "calc_result",
                "schema": CalcResult.model_json_schema(),
            }
        }
    }
)

In [13]:
print(calculator_agent.model)

magistral-medium-latest


In [20]:
# Define a new variable for the calculator query
calculator_query = "What is 125 times 34 divided by 5?"
print(f"\n--- Test Case for Calculator Agent ---")
print(f"\nUser: {calculator_query}")

# Create a new conversation history list for the calculator test case
calculator_conversation_history = [{"role": "user", "content": calculator_query}]

# Call client.chat.complete using the calculator_agent.model
print("[DEBUG] Sending user query to the calculator_agent...")
try:
    response_calculator = client.chat.complete(
        model=new_calculator_agent.model, # Use the model associated with the calculator_agent
        messages=calculator_conversation_history,
        # No 'tools' list needed here because the agent is configured for a structured response format
    )

    assistant_message_calculator = response_calculator.choices[0].message
    print("\nCalculator Agent's Response:")

    # Process the response. Check if the response content can be parsed as JSON.
    response_content = assistant_message_calculator.content
    try:
        # If the response is JSON, parse it and print the reasoning and result fields
        parsed_response = json.loads(response_content)
        # Assuming the JSON structure matches the CalcResult BaseModel
        if isinstance(parsed_response, dict) and "reasoning" in parsed_response and "result" in parsed_response:
            print(f"  Reasoning: {parsed_response['reasoning']}")
            print(f"  Result: {parsed_response['result']}")
            # Append the parsed content (or a representation of it) to history
            calculator_conversation_history.append({"role": "assistant", "content": json.dumps(parsed_response)}) # Append JSON string
        else:
            # If it's JSON but doesn't match the expected structure
            print("  Response is JSON but does not match CalcResult structure.")
            print("  Raw Response Content:")
            print(response_content)
            # Append the raw content to history
            calculator_conversation_history.append({"role": "assistant", "content": response_content})

    except json.JSONDecodeError:
        # If the response is not JSON, print the raw response content
        print("  Response is not in JSON format.")
        print("  Raw Response Content:")
        print(response_content)
        # Append the raw content to history
        calculator_conversation_history.append({"role": "assistant", "content": response_content})

except Exception as e:
    print(f"\nAn error occurred during calculator agent interaction: {e}")
    # Append an error message to history (optional, for debugging)
    calculator_conversation_history.append({"role": "assistant", "content": f"Error: {e}"})


# Print a message indicating the completion of the calculator test case.
print("\n--- Calculator Test case execution complete. ---")



--- Test Case for Calculator Agent ---

User: What is 125 times 34 divided by 5?
[DEBUG] Sending user query to the calculator_agent...

Calculator Agent's Response:
  Response is not in JSON format.
  Raw Response Content:
<think>
Alright, I have a problem to solve: "What is 125 times 34 divided by 5?"

First, I need to understand the expression. It's a combination of multiplication and division. The expression is 125 * 34 / 5.

### Step 1: Break Down the Problem
I know that multiplication and division have the same precedence, so they are evaluated from left to right. But, according to the order of operations (PEMDAS/BODMAS), multiplication and division are performed from left to right. So, I can rewrite the expression to make it clearer:

\[ \frac{125 \times 34}{5} \]

But, I recall that division can be distributed over multiplication, but since it's all in one expression, I might want to see if I can simplify before multiplying large numbers.

### Step 2: Simplify the Expression
Lo